In [37]:
import sched, time
import yfinance as yf
import cryptocompare as cp

In [7]:
import requests as r, json
from getpass import getpass

In [5]:
url = 'https://bigdataforall.com/crypto/?'

In [8]:
print('Enter your bitcoin account key')
key = getpass()

Enter your bitcoin account key
········


In [92]:
profile_url = f"{url}key={key}&request=profile"
reset_url =  f"{url}key={key}&request=reset"
buy_url = f"{url}key={key}&request=buy&symbol={sym}&quantity={qty}"
sell_url = f"{url}key={key}&request=sell&symbol={sym}&quantity={qty}"

In [102]:
## reset data
response = r.get(reset_url).json()
## set initial params
qty = 0.1  ## qty planning to buy 
sym = "BTC" 
p_qty = 0 ## purchased qty

In [103]:
response

{'username': 'jodie',
 'cash': 100000,
 'shares value': 0,
 'penalty': 0,
 'total': 100000,
 'shares': []}

In [104]:

while True:
    print ("checking balance and bitcoin price...")
    ## get the current BTC price and current account profile
    BTC = cp.get_price('BTC',currency='USD')['BTC']['USD']
    response = r.get(profile_url).json()
    cash = response['cash']
    total = response['total']
    b_asset = total - cash
    
    print(f"Cash = {cash} , Total = {total}, purchased bitcoin value = {b_asset} /// Bitcion price = {BTC}")
    
    if p_qty == 0: ## first time buy -- buy 20%
        qty = cash/5/BTC
        buy_url = f"{url}key={key}&request=buy&symbol={sym}&quantity={qty}"
        response = r.get(buy_url).json()
        p_qty = p_qty + qty
        b_cost = cash/5  
        avg_cost = b_cost / p_qty
        print(f"purchased {qty} coins, now total own {p_qty} coins")
        
    elif avg_cost < BTC: ## if making money: sell 25% owned bitcoin
        qty = b_asset/4/BTC
        sell_url = f"{url}key={key}&request=sell&symbol={sym}&quantity={qty}"
        response = r.get(sell_url).json()
        p_qty = p_qty - qty
        b_cost = b_cost- (avg_cost*qty)
        print(f"sold {qty} coins, now total own {p_qty} coins")
        
    elif avg_cost > BTC:  ## if not making money: buy 12.5% cash value bitcoin
        qty = cash/8/BTC
        buy_url = f"{url}key={key}&request=buy&symbol={sym}&quantity={qty}"
        response = r.get(buy_url).json()
        p_qty = p_qty + qty
        b_cost = b_cost + (cash/8)
        avg_cost = b_cost / p_qty
        print(f"purchased {qty} coins, now total own {p_qty} coins")
        
    else:
        print("no action")
    
    
    time.sleep(28)

checking balance and bitcoin price...
Cash = 100000 , Total = 100000, purchased bitcoin value = 0 /// Bitcion price = 55076.34
purchased 0.3631323359540594 coins, now total own 0.3631323359540594 coins
checking balance and bitcoin price...
Cash = 80004.909549182 , Total = 100000.67179482, purchased bitcoin value = 19995.76224563799 /// Bitcion price = 55091.81
sold 0.09073836131739904 coins, now total own 0.2723939746366604 coins
checking balance and bitcoin price...
Cash = 85001.508153486 , Total = 100010.41615597, purchased bitcoin value = 15008.908002484008 /// Bitcion price = 55088.78
sold 0.06811236336366501 coins, now total own 0.20428161127299538 coins
checking balance and bitcoin price...
Cash = 88752.638545057 , Total = 100025.85389304, purchased bitcoin value = 11273.215347983001 /// Bitcion price = 55099.71
sold 0.05114915916972613 coins, now total own 0.15313245210326926 coins
checking balance and bitcoin price...
Cash = 91576.170849103 , Total = 100039.44847154, purchased 

KeyboardInterrupt: 

In [46]:
# test -- 
BTC = cp.get_price('BTC',currency='USD')['BTC']['USD']

In [47]:
BTC

54691.59